In [1]:
# Food Delivery Data Analysis
#Innomatics Hackathon Submission

'''This notebook demonstrates loading, merging, and analyzing food delivery
data from CSV, JSON, and SQL sources.'''


'This notebook demonstrates loading, merging, and analyzing food delivery\ndata from CSV, JSON, and SQL sources.'

In [7]:
import pandas as pd
import sqlite3
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


The orders dataset contains transactional order-level information.

In [8]:
#The users dataset contains user details including city and membership type.

users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [9]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


Restaurant data is loaded from an SQL file using SQLite in-memory database.


In [10]:
merged_df = orders.merge(users, on="user_id", how="left")
merged_df = merged_df.merge(restaurants, on="restaurant_id", how="left")

merged_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


Datasets are merged using LEFT JOIN to retain all order records.


In [11]:
merged_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [12]:
# 1. City with Highest Total Revenue from Gold Members
gold_city_revenue = (
    merged_df[merged_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [13]:
# 2. Cuisine with Highest Average Order Value
merged_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [14]:
#3. Users with Total Order Value Greater Than ₹1000
user_spend = merged_df.groupby("user_id")["total_amount"].sum()
(user_spend > 1000).sum()


np.int64(2544)

In [15]:
# 4. Restaurant Rating Range Generating Highest Revenue
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

merged_df["rating_range"] = pd.cut(merged_df["rating"], bins=bins, labels=labels)

merged_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-1221632645.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [16]:
#5. Highest Average Order Value City (Gold Members)
(
    merged_df[merged_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [18]:
# 6. Cuisine with Fewest Restaurants but Significant Revenue
merged_df.groupby("cuisine")["restaurant_id"].nunique().sort_values()
merged_df.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


In [19]:
#total gold orders
merged_df[merged_df["membership"] == "Gold"].shape[0]


4987

In [20]:
#total revenue from Hyderabad
round(
    merged_df[merged_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [21]:
#Distinct users
merged_df["user_id"].nunique()


2883

In [22]:
#Average order Value
round(
    merged_df[merged_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [23]:
#orders with rating >= 4.5

merged_df[merged_df["rating"] >= 4.5].shape[0]


3374

In [24]:
#orders in top gold-revenue city
top_city = gold_city_revenue.index[0]

merged_df[
    (merged_df["membership"] == "Gold") &
    (merged_df["city"] == top_city)
].shape[0]



1337

# Food Delivery Data Analysis  
### Innomatics Hackathon Submission

**Name:** P.Pavani  
**Institute:** Vignan’s Institute of Information Technology  
**Branch:** CSE – Artificial Intelligence  

This notebook analyzes food delivery data by combining CSV, JSON, and SQL
datasets to answer business and analytical questions provided in the hackathon.
